# KillSwitch Project: Design Candidate Guide RNAs

Author: Brandon Holt

This is a cleaned-up and fixed version of [5_KillSwitch_PAM_candidates](5_KillSwitch_PAM_candidates.ipynb), using the correct primer design and only 2 PAM sites.

## Choose an essential gene

Used Yeastmine to search for genes whose "Gene Phenotype Summary" said they were an "Essential gene". Query code in python below:

In [ ]:
from IPython.display import HTML
from intermine.webservice import Service
service = Service("http://yeastmine.yeastgenome.org/yeastmine/service")
query = service.new_query("Gene")
query.add_view(
    "primaryIdentifier", "secondaryIdentifier", "organism.shortName", "symbol",
    "name", "phenotypeSummary"
)
query.add_constraint("phenotypes.strainBackground", "=", "W303", code = "A")
query.add_constraint("phenotypeSummary", "LIKE", "Essential gene*", code = "B")

columns = ["primaryIdentifier", "secondaryIdentifier", "organism.shortName", "symbol", "name", "phenotypeSummary"]

output = '<html><table>'
output += '<tr>' + ' '.join(['<th>'+k+'</th>' for k in columns]) + '</tr>'
for row in query.rows():
    output += '<tr>' + ' '.join(['<td>'+str(row[c])+'</td>' for c in columns]) + '</tr>'
output += '</table></html>'
HTML(output)

Out of those results, I selected the following 2 genes:

- **[LCB1 / YMR296C](http://www.yeastgenome.org/locus/S000004911/overview):** This is responsible for sphingoid synthesis, defects in this gene result in inviablity. I picked it because it said it talks about "shmoo formation" which sounded cool.
- **[RPT5 / YOR117W](http://www.yeastgenome.org/locus/S000005643/overview):** This is one of the 6 ATPases, mutations cause short lifespan, shorter telomeres.

In [58]:
import coral

# Guide RNA Design for RPT5

## Find candidate PAM sites.
Download promoter & gene sequences and label their features. Then do a search for '_GG' in the promoter.

In [3]:
rpt5_promoter = coral.database.get_yeast_promoter_ypa('RPT5')
print 'promoter length:', len(rpt5_promoter)
rpt5_promoter.name = 'rpt5-Promoter'
rpt5_promoter.features.append(coral.Feature('rpt5-Promoter', 0, len(rpt5_promoter), 'promoter'))

rpt5_cds = coral.database.fetch_yeast_locus_sequence('rpt5')
rpt5_cds.name = 'RPT5'
rpt5_cds.features.append(coral.Feature('RPT5', 0, len(rpt5_cds), 'CDS'))

# find 'NGG' sites, list the last 10 in the promoter (forward direction)
all_sites = rpt5_promoter.locate('GG')
sites = all_sites[0][-10:]
sites

promoter length: 883


[719, 722, 756, 777, 778, 786, 787, 796, 845, 859]

Choose sites. Because the 2nd overlaps with the 1st, let's do the 3rd site. Verify that they are followed by NGG:

In [31]:
site1 = sites[-1]-1
rpt5_g1 = rpt5_promoter[site1-20:site1]
print 'site #1:', rpt5_g1, rpt5_promoter[site1:site1+3]

site2 = sites[-3]-1
rpt5_site2 = site2
rpt5_g2 = rpt5_promoter[site2-20:site2]
print 'site #2:', rpt5_g2, rpt5_promoter[site2:site2+3]

# create sequences and add features for visualizing it
rpt5_promoter.features.append(coral.Feature('rpt5-pam1', site1, site1, 'site'))
rpt5_g1.name = 'rpt5-guide1'
rpt5_g1.features.append(coral.Feature('rpt5-guide1', 0, 20, 'misc_RNA'))

rpt5_promoter.features.append(coral.Feature('rpt5-pam2', site2, site2, 'site'))
rpt5_g2.name = 'rpt5-guide2'
rpt5_g2.features.append(coral.Feature('rpt5-guide2', 0, 20, 'misc_RNA'))

site #1: TTATAGAGGTGAGAACAAAT TGG
site #2: TTGGGTAAAAAGGGCACATC AGG


## Make mockups

In [38]:
# import template plasmid
orig_plasmid = coral.seqio.read_dna('./pmod8-pgalz4-urgr.gb')
target = orig_plasmid.select_features('Target Sequence')[0]
print 'target seq:', orig_plasmid[target.start:target.stop]

# helper functions for displaying part of the plasmid

def feat(dna, name):
    return dna.select_features(name)[0]

def zoom(plasmid):
    # need to rotate so the zoomed region doesn't wrap around 0
    plasmid = plasmid.rotate(200)
    zoom = (feat(plasmid,'ASBV1').start, feat(plasmid,'gRNA Handle').stop)
    pz = plasmid[zoom[0]:zoom[1]]
    pz.name += ' (zoom ASBV1..gRNA)'
    return pz.display()

# linearize backbone by removing target sequenceb
backbone = orig_plasmid.rotate(target.start)[target.stop-target.start:]

# display the whole plasmid
orig_plasmid.display()

target seq: NNNNNNNNNNNNNNNNNNNN


In [39]:
# make mockups
rpt5_g1_mockup = (rpt5_g1+backbone).circularize()
rpt5_g1_mockup.name = 'pMOD6-killswitch-ASBV1-g1'
zoom(rpt5_g1_mockup) # need to rotate so the zoomed region doesn't wrap around 0

In [40]:
rpt5_g2_mockup = (rpt5_g2+backbone).circularize()
rpt5_g2_mockup.name = 'pMOD6-killswitch-ASBV1-g2'
zoom(rpt5_g2_mockup)

## Design primers
Find homology with each end of the backbone that anneals at 65°C, then use `coral.design.OligoAssembly` to find primers for the fragment containing our guides and the homology.

In [59]:
def design_primers(guide):
    # find regions that anneal at 65°C on both ends of backbone
    h5prime = coral.design.primer(backbone.flip(), tm=65).primer().to_ds().flip()
    h3prime = coral.design.primer(backbone, tm=65).primer().to_ds()
    
    # construct desired fragment with guide and homology with backbone on both sides
    fragment = h5prime + guide + h3prime
    
    # design primers to generate the entire fragment
    o = coral.design.OligoAssembly(fragment, tm=65, length_range=(20,60))\
             .design_assembly()
    
    fwd = coral.Primer(o['oligos'][0], tm=o['overlap_tms'][0])
    rev = coral.Primer(o['oligos'][1], tm=o['overlap_tms'][0])
    
    return (fragment, fwd, rev)


rpt5_g1_fragment, rpt5_g1_fwd, rpt5_g1_rev = design_primers(rpt5_g1)
rpt5_g2_fragment, rpt5_g2_fwd, rpt5_g2_rev = design_primers(rpt5_g2)

print 'site #1 fragment:'
print ' ', rpt5_g1_fragment
print 'site #1 primers:'
print ' ', rpt5_g1_fwd, 'len:', len(rpt5_g1_fwd)
print ' ', rpt5_g1_rev, 'len:', len(rpt5_g1_rev)
print
print 'site #2 fragment:'
print ' ', rpt5_g2_fragment
print 'site #2 primers:'
print ' ', rpt5_g2_fwd, 'len:', len(rpt5_g2_fwd)
print ' ', rpt5_g2_rev, 'len:', len(rpt5_g2_rev)

site #1 fragment:
  GCCCGTCCCATCTCCCTTATAGAGGTGAGAACAAATGTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGG
site #1 primers:
  GCCCGTCCCATCTCCCTTATAGAGGTGAGAACAAATGTTTTAGAGCTAGA len: 50
  CCTTATTTTAACTTGCTATTTCTAGCTCTAAAACATTTGTTCTCACCTCT len: 50

site #2 fragment:
  GCCCGTCCCATCTCCCTTGGGTAAAAAGGGCACATCGTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGG
site #2 primers:
  GCCCGTCCCATCTCCCTTGGGTAAAAAGGGCACATCGTTTTAGAGCT len: 47
  CCTTATTTTAACTTGCTATTTCTAGCTCTAAAACGATGTGCCCTTT len: 46


### Validate against mockups
Our goal is to verify that we can do a Gibson with the 2 fragments from Aquarium and our fragments and get back our mockup. The Aquarium samples don't have the sequence so I generate the sequence by taking the primers used to generate the Aquarium fragments and simulate running PCR with source plasmid.

In [57]:
# Primers linked from Fragment 1 on Aquarium (http://54.68.9.194:81/samples/3351)
frag1_primers = (
    coral.Primer(anneal=coral.DNA('gttttagagctagaaatagcaagttaaaataaggctagtccg'), tm=71.4),
    coral.Primer(anneal=coral.DNA('ATCGGAGGACCGAAGGAGCTAACC'), tm=72)
)
frag1 = coral.reaction.pcr(orig_plasmid, *frag1_primers)

# Primers linked from Fragment 2 on Aquarium (http://54.68.9.194:81/samples/5540)
frag2_primers = (
    coral.Primer(anneal=coral.DNA('GGGAGATGGGACGGGCG'), tm=71.86),
    coral.Primer(anneal=coral.DNA('GTTCGCCAGTTAATAGTTTGCGCAACG'), tm=72)
)
frag2 = coral.reaction.pcr(orig_plasmid, *frag2_primers)

print 'Site #1:'
# simulate PCR with our primers to generate our fragment
rpt5_g1_pcr = coral.reaction.pcr(rpt5_g1_fwd.primer().to_ds(), rpt5_g1_fwd, rpt5_g1_rev)
if rpt5_g1_pcr == rpt5_g1_fragment:
    print ' - Validated primers against mockup fragment.'

# simulate gibson with 2 aquarium fragments and our simulated pcr fragment
rpt5_g1_gibson = coral.reaction.gibson([frag1, frag2, rpt5_g1_pcr])
if rpt5_g1_gibson.is_rotation(rpt5_g1_mockup):
    print ' - Validated Gibson with Aquarium fragments.'

print 'Site #2:'
# simulate PCR with our primers to generate our fragment
rpt5_g2_pcr = coral.reaction.pcr(rpt5_g2_fwd.primer().to_ds(), rpt5_g2_fwd, rpt5_g2_rev)
if rpt5_g2_pcr == rpt5_g2_fragment:
    print ' - Validated primers against mockup fragment.'

# simulate gibson with 2 aquarium fragments and our simulated pcr fragment
rpt5_g2_gibson = coral.reaction.gibson([frag1, frag2, rpt5_g2_pcr])
if rpt5_g2_gibson.is_rotation(rpt5_g2_mockup):
    print ' - Validated Gibson with Aquarium fragments.'

Site #1:
 - Validated primers against mockup fragment.
 - Validated Gibson with Aquarium fragments.
Site #2:
 - Validated primers against mockup fragment.
 - Validated Gibson with Aquarium fragments.
